### Recovering data from SQL SB

In [52]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt

pd.options.display.max_columns = 999
%matplotlib inline

In [2]:
conn = psycopg2.connect("dbname = soaring_predictor")

### Loading flying metrics

In [3]:
F = pd.read_sql('SELECT * from flight_metrics;', conn)

In [21]:
F.columns

Index(['flight', 'date', 'pilot', 'launch_time', 'landing_time', 'flying_time',
       'launch_coord', 'landing_coord', 'landed_lz', 'linear_dist', 'max_alt',
       'total_alt_gain', 'left_perimiter', 'month', 'year', 'alt_sampe',
       'len_sampe', 'doy', 'timestamp'],
      dtype='object')

In [23]:
# Isolating the best flight of the date (by max flying time)

Y = pd.DataFrame(F.groupby('date')['flying_time','max_alt','left_perimiter'].max())

### Loading weather data

In [10]:
W = pd.read_sql('SELECT * from weather;', conn)

### Merging the 2 sources of data

In [25]:
data = pd.merge(Y,W,how='left', right_on='date',left_index=True)

In [26]:
data.dropna(inplace=True)

In [27]:
import io

from sqlalchemy import create_engine

# fail, append or replace

def write_to_table(df, db_engine, table_name, if_exists='fail'):
    string_data_io = io.StringIO()
    df.to_csv(string_data_io, sep='|', index=False)
    pd_sql_engine = pd.io.sql.pandasSQL_builder(db_engine)
    table = pd.io.sql.SQLTable(table_name, pd_sql_engine, frame=df,
                               index=False, if_exists=if_exists)
    table.create()
    string_data_io.seek(0)
    string_data_io.readline()  # remove header
    with db_engine.connect() as connection:
        with connection.connection.cursor() as cursor:
            copy_cmd = "COPY %s FROM STDIN HEADER DELIMITER '|' CSV" % table_name
            cursor.copy_expert(copy_cmd, string_data_io)
        connection.connection.commit()

In [28]:
address = 'postgresql://@localhost:5432/soaring_predictor'
engine = create_engine(address)

In [29]:
write_to_table(data, engine, 'data',if_exists='replace')

In [30]:
data.columns

Index(['flying_time', 'max_alt', 'left_perimiter', 'date', 'alti_0', 'alti_4',
       'alti_8', 'alti_12', 'alti_16', 'alti_20', 'drct_0', 'drct_4', 'drct_8',
       'drct_12', 'drct_16', 'drct_20', 'dwpf_0', 'dwpf_4', 'dwpf_8',
       'dwpf_12', 'dwpf_16', 'dwpf_20', 'p01i_0', 'p01i_4', 'p01i_8',
       'p01i_12', 'p01i_16', 'p01i_20', 'relh_0', 'relh_4', 'relh_8',
       'relh_12', 'relh_16', 'relh_20', 'sknt_0', 'sknt_4', 'sknt_8',
       'sknt_12', 'sknt_16', 'sknt_20', 'skyc1_0', 'skyc1_4', 'skyc1_8',
       'skyc1_12', 'skyc1_16', 'skyc1_20', 'tmpf_0', 'tmpf_4', 'tmpf_8',
       'tmpf_12', 'tmpf_16', 'tmpf_20', 'doy'],
      dtype='object')

In [31]:
data.to_csv('data.csv')

In [20]:
pwd

'/Users/eduardodeangelis/Desktop/galvanize/soaring-predictor/notebooks'

In [33]:
T = pd.DataFrame(W.groupby('date')['tmpf_12'].mean())

In [45]:
# T.reset_index(inplace=True)
T.drop('level_0',inplace=True,axis=1)

In [47]:
W = pd.merge(Y,T,how='left',right_on='date',left_index=True)

In [49]:
W.dropna(inplace=True)

In [55]:
# returns day of the year

def doy(dt_obj):
    return int(dt_obj.strftime('%j'))

In [57]:
W['doy']=W['date'].apply(doy)

In [65]:
# fig,ax = plt.subplots(figsize = (18,8))
# ax.scatter(W['doy'], W['flying_time'], alpha = 0.5,)
# ax.plot(W['doy'],W['tmpf_12'])
# ax.set_title('Flight time [in minutes]',fontsize=20)
# ax.set_ylabel('minutes', fontsize=16)
# ax.set_xticks([0,90,180,270]) # choose which x locations to have ticks
# ax.set_xticklabels(['Jan','Apr',"Jul",'Oct'], fontsize=14)

In [64]:
W.sort_values('doy')
# W.groupby('doy')['tmpf_12'].mean()

,flying_time,max_alt,left_perimiter,date,tmpf_12,doy
366,16.583333,500,0,2014-01-03,45.005000,3
367,16.266667,546,0,2014-01-04,43.295000,4
9,21.366667,493,0,2013-01-11,34.790000,11
10,23.066667,682,0,2013-01-12,36.005000,12
740,7.166667,563,0,2015-01-12,50.990000,12
14,16.616667,478,0,2013-01-16,39.020000,16
748,20.250000,563,0,2015-01-20,48.470000,20
384,8.150000,517,0,2014-01-21,44.735000,21
1482,61.500000,625,1,2017-01-23,45.903269,23
386,15.333333,554,0,2014-01-23,46.715000,23
